# Adversarial Machine Learning in Network Intrusion Detection Systems

L'article utilise comme dataset NSL-KDD et UNSW-NB15, qui représente des flux de données bruts. Ces flux sont un mélanges différent type de trafic mais aussi un mélange  de bonnes et malicieuses données. Le but étant d'altérer ces paquets de données pour pouvoir passer les modèles de machine learning de sécurité tout en gardant en gardant les flux de donnée fonctionnels. 

In [13]:
import os
import pandas as pd


df = pd.read_csv("KDDTest+.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22544 entries, 0 to 22543
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     22544 non-null  int64  
 1   protocol_type                22544 non-null  object 
 2   service                      22544 non-null  object 
 3   flag                         22544 non-null  object 
 4   src_bytes                    22544 non-null  int64  
 5   dst_bytes                    22544 non-null  int64  
 6   land                         22544 non-null  int64  
 7   wrong_fragment               22544 non-null  int64  
 8   urgent                       22544 non-null  int64  
 9   hot                          22544 non-null  int64  
 10  num_failed_logins            22544 non-null  int64  
 11  logged_in                    22544 non-null  int64  
 12  num_compromised              22544 non-null  int64  
 13  root_shell      

In [14]:
df2 = pd.read_csv("UNSW_NB15_training-set.csv")
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  object 
 3   service            175341 non-null  object 
 4   state              175341 non-null  object 
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

# Genetic Algorithm

Le *Genetic algorithm* (GA) est un algorithme qui se base sur la génétique et la sélection naturelle. Le GA utilise une forme de données appelées chromosome : ces données (vecteurs) sont formées et les valeurs de différentes features sont destinées à évoluer comme pour un gêne d'être vivant.
Certaines *features* sont inchangeable du à leur importance dans le bon fonctionnement des paquets.

- En premier lieu, on va créer une population de 100 chromosomes, dont leurs données seront générer aléatoirement, séparées en 2 parties, les données que l'ont peu modifier et les autres. (pour les opérations qui suivent, on va utiliser la partie que l'on peut modifier)
- on va ensuite utiliser l'opération de *cross over*. On choisi aléatoirement 2 chromosomes que l'on va séparer l'un de 25% de ses features et donc, de l'autre des 75% autres, que l'on va échanger pour créer 2 nouveau chromosomes. Ces nouveaux chromosomes seront ensuite insérés dans la population.
- Enfin, on va utiliser l'opération de *mutation* sur la partie que l'on peut modifier, qui consiste à choisir de manière aléatoire un chromosome et de même pour une de ses features. Toute cette opération est exécutée 
- On regroupe les 2 parties des chromosomes.
- Finalement, on va utiliser une *fitness function* pour évaluer chaque chromosome, s'ils sont bénins (seuil à 99,99%) ou non. On retiendra le meilleur chromosome toutes les 5 générations.
- On répète tout ce processus 100 fois
On obtient alors un nouveau dataset ne contenant que des vecteurs "malsains" qui seront testés dans la partie expérience.

In [18]:
import sklearn
import numpy as np
import random
from random import randint
from geneticalgorithm import geneticalgorithm as ga
from sklearn import svm
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

#Creation of initial population
listProtocols = ["tcp","udp","icmp"]
listData = ["private","ftp_data","eco_i","telnet","http","smtp","imap4","systat","pop3","domain_u","whois","netbios_dgm"]
listSF = ["SF","S0","REJ","RSTR"]
#cell 3 = 62825648 - 0
# 24 à 30 et 33 à 40 entre 0 et 1
# 4 à 23 , 31- 32,  entre 0 et 20
#derniere = 0 - 21

# creation of artifical dataset
def initialPopulation(df, nbChrom):
    initPopulation = []
    for i in range (nbChrom):
        chrom = []
        for j in range (len(df.columns)):
            if(j == 0) :
                chrom.insert(j,random.randint(0, 2))
            if(j == 1) :
                chrom.insert(j,random.randint(0, 11))
            if(j == 2) :
                chrom.insert(j,randint(0, 62825648))
            if((j >= 24 and j <= 30) or (j >= 33 and j <= 40)) :
                chrom.insert(j,random.uniform(0, 1))
            if((j >= 4 and j <= 23) or j == 32 or j ==33) :
                chrom.insert(j,random.randint(0, 62825648))
            if(j == 41) :
                chrom.insert(j,random.randint(0,21))
        initPopulation.append(chrom)
    return initPopulation




dimDF = len(df.columns)
dimDF2 = len(df2.columns)

#We have to reduce to a minimum data size otherwise our computers can not run the fit after...
dfCopy = df.copy()
dfCopy = dfCopy.head(1000)

# preparation of the data : transforming string input to int 
LE = LabelEncoder()
dfCopy['protocol_type'] = LE.fit_transform(dfCopy['protocol_type'])
dfCopy['service'] = LE.fit_transform(dfCopy['service'])
dfCopy['flag'] = LE.fit_transform(dfCopy['flag'])
dfCopy['class'] = LE.fit_transform(dfCopy['class'])
dfCopy.head()

y = dfCopy['class']
dfCopy.drop(['class'], axis='columns', inplace=True)
X = dfCopy



In [19]:

population = initialPopulation(dfCopy,100)
#fitnessScore(population,)
#population

In [17]:
#Parameters given in the section 4.3 of the article
svm = svm.SVC(C = 220, gamma = 0.01, probability = True, tol = 0.001)
#  decision trees
dectr = tree.DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 4, \
                                    min_samples_leaf = 2, max_depth = 20, min_impurity_decrease = 0.1)
#naive bayes
nb = GaussianNB()
#k nearest neighbors
knn = KNeighborsClassifier(n_neighbors = 3, algorithm ='auto')

estimators = [ ('svm', svm), ('dt', dectr), ('nb', nb), ('knn', knn)]
model = ensemble.VotingClassifier(estimators, voting='hard')

fitModel = model.fit(X,y)


# https://datascienceplus.com/genetic-algorithm-in-machine-learning-using-python/

def fitnessScore(population,yTest):
    scores = []
    for chromosome in population :
        prediction = fitModel.predict(chromosome)
        # We want the bad pred to be valued better
        # 0 correspond to "anomaly" and 1 to "normal"
        if(prediction == 0):
            score = 1
        else :
            score = 0
        scores.append(score)
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

''' # TO ADAPT
def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen

def crossover(pop_after_sel):
    population_nextgen=pop_after_sel
    for i in range(len(pop_after_sel)):
        child=pop_after_sel[i]
        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen

def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

def generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, y_train, y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print(scores[:2])
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score
'''

' # TO ADAPT\ndef selection(pop_after_fit,n_parents):\n    population_nextgen = []\n    for i in range(n_parents):\n        population_nextgen.append(pop_after_fit[i])\n    return population_nextgen\n\ndef crossover(pop_after_sel):\n    population_nextgen=pop_after_sel\n    for i in range(len(pop_after_sel)):\n        child=pop_after_sel[i]\n        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]\n        population_nextgen.append(child)\n    return population_nextgen\n\ndef mutation(pop_after_cross,mutation_rate):\n    population_nextgen = []\n    for i in range(0,len(pop_after_cross)):\n        chromosome = pop_after_cross[i]\n        for j in range(len(chromosome)):\n            if random.random() < mutation_rate:\n                chromosome[j]= not chromosome[j]\n        population_nextgen.append(chromosome)\n    #print(population_nextgen)\n    return population_nextgen\n\ndef generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,\n                                  

Le but est de maximiser le nombre de vecteurs malsain classés en sain.

# Particle Swarn Optimization

L'idée du *Particle Swarm Optimization* est que pour chaque particule d'une nuée, on va essayer de trouver la meilleure position retournée par la *fitness function*. 

- D'abord, on doit créer la nuée de 200 particules. Les particules sont formées à partir d'une rangée du dataset et d'une partie généré aléatoirement, toujours avec les mêmes *features* que le dataset. Comme que pour le GA, on devrait prendre seulement la partie mutable des particules à modifier. Ensuites, pour chaque particules est attribué une vélocité de 0.7.
- Pour chaque itération, on va évaluer chaque particule à partir de la *fitness function*. On met à jour la meilleur valeur de la fonction fitness possible de chaque particules (de 0.5 au départ) et de la meilleur valeur de la fonction fitness global (de 0.4 au départ)
- On calcule en suite la distance entre la position de chaque particule et on récupère la meilleure valeur grâce à la fonction de fitness. On met à jour la vélocité et la position de chaque particule avec les valeurs précédentes.
- On répète tout ce processus 100 fois au maximum ou s'il y a une amélioration de moins de 0.001%.
La nué de particules devient alors un nouveau dataset de vecteurs *malsains*

In [59]:
!pip install pyswarms

In [60]:
from pyswarms.single import GlobalBestPSO
from pyswarms.utils.search.grid_search  import SearchBase

from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

def prepare_UNSW_NB15(data):
    LE = LabelEncoder()
    copy = data[data['label']==1]
    copy = copy.drop(['label'], axis=1)
    copy['proto'] = LE.fit_transform(copy['proto'])
    copy['service'] = LE.fit_transform(copy['service'])
    copy['state'] = LE.fit_transform(copy['state'])
    copy['attack_cat'] = LE.fit_transform(copy['attack_cat'])
    return copy

def voting_classifier(X_train, Y_train) :
  sv = svm.SVC(gamma = 0.01, C = 220.0, tol = 0.01, probability = True)
  dt = DecisionTreeClassifier(
      criterion = "entropy", min_samples_split = 4, min_samples_leaf = 2, max_depth = 20, min_impurity_decrease = 0.1)
  nb = GaussianNB()
  kn = KNeighborsClassifier( n_neighbors = 3, algorithm = "auto")

  vC = VotingClassifier (estimators=
                         [('svm', sv), ('dt', dt), ('nb', nb), ('knn',kn)], voting='soft')
  vC = vC.fit(X_train,Y_train)
  return vC

def Pso (dim, data) :
  option = {'c1': 0.5, 'c2': 0.4, 'w' : 0.7}
  opt = GlobalBestPSO (n_particles=200, dimensions=dim, options=option)
  return opt.optimize(prepare_UNSW_NB15, 100, None, True, data)

In [61]:
from sklearn.model_selection import train_test_split

df2prepare = Pso (len(copy.columns), df2)

#X_train, X_test, y_train, y_test = train_test_split(df2prepare, test_size=0.2, random_state=300)

#voting_classifier(X_train, y_train)

2020-11-25 10:30:06,933 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.4, 'w': 0.7}
pyswarms.single.global_best:   0%|          |0/100


TypeError: voting_classifier() got multiple values for argument 'X_train'

# Generative Adversarial Network

Le *Generative Adversial Network* est une technique de deep learning qui va confronter deux réseaux de neuronnes. Ces deux réseaux de neuronnes correspondent l'un à la génération des vecteur *malicieux* et donc à ceux qu'elles permettent de tromper, l'autre est le discriminateur, qui lui va essayer de deviner si l'entrée est saine ou non.
Comme les méthodes précédentes, on va modifier d'abord la partie modifiable, c'est à dire non fixée des vecteurs. 
- Pour le générateur, on va sélectionner aléatoirement un vecteur "malsain". On va ajouter du bruit dans certaines des features choisies au hasard, toujours dans la partie modifiable du vecteur malicieux. Puis ce vecteur va rentrer dans le réseau neuronal du générateur avec comme sortie, un vecteur de la taille de la partie mutable du vecteur d'entrée. On recombine le vecteur de sortie avec la partie non changeable du vecteur de l'entrée, et on le donne en entrée du discriminateur. Les labels sortant de ce dernier permettrons d'améliorer le réseau de neurone du générateur.
- Pour le discriminateur, on va faire la même chose avec mais un vecteur bénin.
- Les deux réseaux de neurones utiliserons des sets de données d'entraînements.



# EXPERIENCE

Les expériences sont enfin réalisée sur les 11 modèles de classification en utilisant les sets produits précédemment. 
L'article ne mentionne pas les hyperparamètres à appliquer pour les 11 classifieurs. 
Les 11 classifieurs sont Support Vector Machine (SVM), Decision Trees (DT), Naive Bayes (NB), k Nearest Neighbors (KNN), Random Forest (RF), Multi-layer Perceptron (MLP), Gradient Boosting (GB), Logistic Regression (LR), Linear Discriminant Analysis (LDA), Quadratic Discriminant Analysis (QDA), and Bagging (BAG)
Nous laisserons principalement les paramètres par défaut pour réaliser les expériences.
Les résultats attendus pour ces expériences sont un taux élevé de mauvaise classification (dans le cas où nous arriverions à générer les adversairal datasets). 
